In [1]:
import glob
import tarfile
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
def read_request_logs(folder, files):
    
    def process_tar_archive(tar_file_name):
        """Read logs from tar archive and assign timestamp to each row by doing file name conversion."""
        with tarfile.open(tar_file_name,'r') as tar:
            tar_log_files = tar.getnames()
            df = pd.concat(
                [
                    pd.read_csv(tar.extractfile(log), parse_dates=True)
                    for log in tar_log_files
                ]
            )
        return df
    
    # Obtain archive file names.
    
    tar_files = glob.glob(folder+files)
    tar_files.sort() # Not mandatory.
    
    # Read logs and assign timestamp to each row by doing file name conversion.
    
    df = pd.concat([process_tar_archive(tar_file) for tar_file in tar_files], sort=False)
    
    for col in filter(lambda s: "S" != s, df.columns):
        df = df.where(df[col] != '-').dropna()
        
    df = df.reset_index()
    
    return df

In [3]:
df = read_request_logs("request_logs/", "*.tar.gz")

/Users/wrath/miniconda3/envs/nesh-monitoring/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [5]:
df.groupby(["Time", "Identifier"])["Jobs"].sum()

Time                        Identifier      
2020-02-06 19:53:10.870544  01a2d1c90f755bb6    1
                            033a31e6d249f4dd    1
                            0370bb67c954f194    1
                            046c2603d1d29c71    6
                            0478207e7bb58efc    1
                                               ..
2020-03-03 11:10:32.432433  fc613da52a78e6a9    1
                            fcb456de189eb32a    1
                            fed9f8a835553808    1
                            ff09dee8306f3da2    4
                            ffa728f0484b5753    1
Name: Jobs, Length: 1566200, dtype: object

In [ ]:
!conda list --explicit